In [ ]:
projectdir = splitpath(pwd()) |>
    ss -> joinpath(ss[1:findall([s == "CounterfactualTraining.jl" for s in ss])[1]]...) 
cd(projectdir)

In [ ]:
using CTExperiments
using CTExperiments.CSV
using CTExperiments.DataFrames
using CTExperiments.StatsBase

using DotEnv
DotEnv.load!()

\FloatBarrier

\setcounter{section}{0}
\renewcommand{\thesection}{\Alph{section}}

\setcounter{table}{0}
\renewcommand{\thetable}{A\arabic{table}}

\setcounter{figure}{0}
\renewcommand{\thefigure}{A\arabic{figure}}

<!-- # Supplementary Material {.appendix} -->

# Notation {.appendix}

- $y^+$: The target class and also the index of the target class.
- $y^-$: The non-target class and also the index of non-the target class.
- $\mathbf{y}^+$: The one-hot encoded output vector for the target class. 
- $\theta$: Model parameters (unspecified).
- $\Theta$: Matrix of parameters. 

# Technical Details of Our Approach {.appendix} 


In [ ]:
#| eval: false

projectdir = splitpath(pwd()) |>
    ss -> joinpath(ss[1:findall([s == "CounterfactualTraining.jl" for s in ss])[1]]...) 
cd(projectdir)

using CTExperiments
using CTExperiments.CounterfactualExplanations
using CTExperiments.CounterfactualTraining
using CTExperiments.Flux
using CTExperiments.Plots
using CTExperiments.TaijaPlotting
using Plots.PlotMeasures

## Protecting Mutability Constraints with Linear Classifiers {#sec-app-constraints}

In @sec-constraints we explain that to avoid penalizing implausibility that arises due to mutability constraints, we impose a point mass prior on $p(\mathbf{x})$ for the corresponding feature. We argue in @sec-constraints that this approach induces models to be less sensitive to immutable features and demonstrate this empirically in @sec-experiments. Below we derive the analytical results in @prp-mtblty.

::: {.proof}

Let $d_{\text{mtbl}}$ and $d_{\text{immtbl}}$ denote some mutable and immutable feature, respectively. Suppose that $\mu_{y^-,d_{\text{immtbl}}} < \mu_{y^+,d_{\text{immtbl}}}$ and $\mu_{y^-,d_{\text{mtbl}}} > \mu_{y^+,d_{\text{mtbl}}}$, where $\mu_{k,d}$ denotes the conditional sample mean of feature $d$ in class $k$. In words, we assume that the immutable feature tends to take lower values for samples in the non-target class $y^-$ than in the target class $y^+$. We assume the opposite to hold for the mutable feature.

Assuming multivariate Gaussian class densities with common diagonal covariance matrix $\Sigma_k=\Sigma$ for all $k \in \mathcal{K}$, we have for the log likelihood ratio between any two classes $k,m \in \mathcal{K}$ [@hastie2009elements]:

$$
\log \frac{p(k|\mathbf{x})}{p(m|\mathbf{x})}=\mathbf{x}^\intercal \Sigma^{-1}(\mu_{k}-\mu_{m})  + \text{const}
$$ {#eq-loglike}

By independence of $x_1,...,x_D$, the full log-likelihood ratio decomposes into:

$$
\log \frac{p(k|\mathbf{x})}{p(m|\mathbf{x})} = \sum_{d=1}^D \frac{\mu_{k,d}-\mu_{m,d}}{\sigma_{d}^2} x_{d} + \text{const}
$$ {#eq-loglike-decomp}

By the properties of our classifier (*multinomial logistic regression*), we have:

$$
\log \frac{p(k|\mathbf{x})}{p(m|\mathbf{x})} = \sum_{d=1}^D \left( \theta_{k,d} - \theta_{m,d} \right)x_d + \text{const}
$$ {#eq-multi}

where $\theta_{k,d}=\Theta[k,d]$ denotes the coefficient on feature $d$ for class $k$. 

Based on @eq-loglike-decomp and @eq-multi we can identify that $(\mu_{k,d}-\mu_{m,d}) \propto (\theta_{k,d} - \theta_{m,d})$ under the assumptions we made above. Hence, we have that $(\theta_{y^-,d_{\text{immtbl}}} - \theta_{y^+,d_{\text{immtbl}}}) < 0$ and $(\theta_{y^-,d_{\text{mtbl}}} - \theta_{y^+,d_{\text{mtbl}}}) > 0$

Let $\mathbf{x}^\prime$ denote some randomly chosen individual from class $y^-$ and let $y^+ \sim p(y)$ denote the randomly chosen target class. Then the partial derivative of the contrastive divergence penalty [@eq-div] with respect to coefficient $\theta_{y^+,d}$ is equal to 

$$
\frac{\partial}{\partial\theta_{y^+,d}} \left(\text{div}(\mathbf{x},\mathbf{x^\prime},\mathbf{y};\theta)\right) = \frac{\partial}{\partial\theta_{y^+,d}} \left( \left(-\mathbf{M}_\theta(\mathbf{x})[y^+]\right) - \left(-\mathbf{M}_\theta(\mathbf{x}^\prime)[y^+]\right) \right) = x_{d}^\prime - x_{d}
$$ {#eq-grad}

and equal to zero everywhere else.

Since $(\mu_{y^-,d_{\text{immtbl}}} < \mu_{y^+,d_{\text{immtbl}}})$ we are more likely to have $(x_{d_{\text{immtbl}}}^\prime - x_{d_{\text{immtbl}}}) < 0$ than vice versa at initialization. Similarly, we are more likely to have $(x_{d_{\text{mtbl}}}^\prime - x_{d_{\text{mtbl}}}) > 0$ since $(\mu_{y^-,d_{\text{mtbl}}} > \mu_{y^+,d_{\text{mtbl}}})$.

This implies that if we do not protect feature $d_{\text{immtbl}}$, the contrastive divergence penalty will decrease $\theta_{y^-,d_{\text{immtbl}}}$ thereby exacerbating the existing effect $(\theta_{y^-,d_{\text{immtbl}}} - \theta_{y^+,d_{\text{immtbl}}}) < 0$. In words, not protecting the immutable feature would have the undesirable effect of making the classifier more sensitive to this feature, in that it would be more likely to predict class $y^-$ as opposed to $y^+$ for lower values of $d_{\text{immtbl}}$. 

By the same rationale, the contrastive divergence penalty can generally be expected to increase $\theta_{y^-,d_{\text{mtbl}}}$ exacerbating $(\theta_{y^-,d_{\text{mtbl}}} - \theta_{y^+,d_{\text{mtbl}}}) > 0$. In words, this has the effect of making the classifier more sensitive to the mutable feature, in that it would be more likely to predict class $y^-$ as opposed to $y^+$ for higher values of $d_{\text{mtbl}}$.

Thus, our proposed approach of protecting feature $d_{\text{immtbl}}$ has the net affect of decreasing the classifier's sensitivity to the immutable feature relative to the mutable feature (i.e. no change in sensitivity for $d_{\text{immtbl}}$ relative to increased sensitivity for $d_{\text{mtbl}}$).

:::

::: {.callout-warning}

\@Cynthia, \@Arie, I have tentatively phrased the above in terms of a theorem and proof. This is something I've so far shied away from because I feel a bit out of my depth when it comes to mathematical proofs. The above makes intuitive sense to me, but I don't know for sure if it's correct. 

:::

In [ ]:
#| eval: false

using CTExperiments: 
    get_ce_data, 
    train_val_split, 
    build_model, 
    LinearModel, 
    get_input_encoder

# Callback:
using CTExperiments: get_log_reg_params, get_decision_boundary

function plot_db(model, ces; _xlab="Debt", _ylab="Age")

    x = (ce -> ce.counterfactual).(ces) |> xs -> reduce(hcat, xs)
    x0 = (ce -> ce.factual).(ces) |> xs -> reduce(hcat, xs)

    xlab = _constraint[1] == "both" ? "$(_xlab) (mutable)" : "$(_xlab) (immutable)"
    ylab = _constraint[2] == "both" ? "$(_ylab) (mutable)" : "$(_ylab) (immutable)"

    # Data and decision boundary:
    coeff = get_log_reg_params(model)
    db = get_decision_boundary(coeff)
    plt = Plots.scatter(
        ce_data, 
        # label=["1=Default" "2=No Default"], 
        # legend_position=:topright, 
        xlab=xlab, ylab=ylab,  
        axis=nothing, 
        legend=false,
        title=_title
    )
    Plots.abline!(plt, db.slope, db.intercept; lw=5, label="Dec. Boundary")

    # Counterfactuals:
    target = 2
    factual = 1
    if !any(isnothing.(x))
        yhat = [argmax(y) for y in eachcol(model(x))]
        yhat0 = [argmax(y) for y in eachcol(model(x0))]
        idx_plotted = yhat0.==factual
        if any(idx_plotted)

            # Paths:
            u = []
            v = []
            for (i,ce) in enumerate(eachcol(x))
                Δ = ce - x0[:,i]
                push!(u, Δ[1])
                push!(v, Δ[2])
            end
            Plots.quiver!(x0[1,idx_plotted], x0[2,idx_plotted], quiver=(u[idx_plotted], v[idx_plotted]), color=factual)
            
            # End points:
            Plots.scatter!(x[1,idx_plotted], x[2,idx_plotted], label=["CE (y⁺=1)" "CE (y⁺=2)"], ms=15, shape=:star, color=yhat[idx_plotted], group=yhat[idx_plotted], mscolor=yhat0[idx_plotted])

        end
    end
    display(plt)
end

# Data:
specs = [
    ("(a)", VanillaObjective(needs_ce=true), ["both", "both"]),
    ("(b)", FullObjective(lambda=[1.0,1.0,0.0,1.0]), ["both", "both"]),
    ("(c)", VanillaObjective(needs_ce=true), ["both", "none"]),
    ("(d)", FullObjective(lambda=[1.0,1.0,0.0,1.0]), ["both", "none"]),
]

In [ ]:
#| eval: false

using CounterfactualExplanations.Convergence
using Random
Random.seed!(42)

models = []
plts = []

for (title, obj, constraint) in specs

    # Globals for the callback:
    global _title = title
    global _constraint = constraint

    # Data:
    data = LinearlySeparable(
        n_train=500,
        batchsize=50,
        mutability=constraint
    )
    global ce_data = get_ce_data(data)
    val_size = data.n_validation / (data.n_validation + data.n_train)
    train_set, val_set, _ = train_val_split(data, ce_data, val_size)

    # Model:
    nin = size(first(train_set)[1], 1)
    nout = size(first(train_set)[2], 1)
    model = build_model(LinearModel(), nin, nout)

    # Objective:
    generator = GenericGenerator()
    opt_state = Flux.setup(Descent(), model)
    conv = MaxIterConvergence(25)

    model, logs = counterfactual_training(
        obj,
        model,
        generator,
        train_set,
        opt_state;
        val_set = val_set,
        nepochs = 100,
        mutability = Symbol.(constraint),
        callback = plot_db,
        nce=50,
        convergence=conv, 
        verbose=3
    )

    push!(models, model)
    push!(plts, current())
end

In [ ]:
#| eval: false

plt = Plots.plot(
    plts..., 
    layout=(1,4), 
    size=(1150,250),  
    left_margin = 10mm, 
    bottom_margin = 5mm,
    top_margin = 3mm,
    right_margin = 10mm,
)
display(plt)
Plots.savefig(plt, "paper/figures/poc.svg")

In [ ]:
#| eval: false

plts = []
titles = ["(a)", "(b)"]
_xlab = "Existing Debt"
_ylab = "Age"
for (i,model) in enumerate(models)
    coeff = get_log_reg_params(model)
    db = get_decision_boundary(coeff)
    xlab = constraints[i][1] == "both" ? "$(_xlab) (mutable)" : "$(_xlab) (immutable)"
    ylab = constraints[i][2] == "both" ? "$(_ylab) (mutable)" : "$(_ylab) (immutable)"
    plt = Plots.scatter(ce_data; xlab=xlab, ylab=ylab, bottom_margin = 5mm, left_margin = 5mm, label=["Default" "No Default"], title=titles[i])
    Plots.abline!(plt, db.slope, db.intercept; lw=5, label="Dec. Boundary")

    push!(plts, plt)
end

plt = Plots.plot(plts..., layout=(1,2), size=(600,300))
Plots.savefig(plt, "paper/figures/app_mtblty.svg")

## Domain Constraints




# Detailed Results {.appendix}

::: {.callout-warning}

\@Cynthia, \@Arie, I'm including some preliminary results here but will rerun experiments in the coming days. You'll notice some odd outlier results (huge values) for the initial grid search (@sec-app-initial). My belief is that this is once again due to *ECCo* overshooting for some hyperparameter choices, that we have discussed before. A simple solution to this story is to actually impose domain constraints. Let's see what the final results show us (I also still plan to make slight changes to the implementation), but in any case I think it might even be worth to report results for the initial grid search in the final appendix (they do include good results for CT for certain hyperparameter ranges and highlight limitations inherited from energy-based modelling).

:::

## Qualitative Findings for Image Data

::: {.callout-note}

\@Cynthia, \@Arie, @fig-mnist shows much more plausible (faithful) counterfactuals for a model with CT than the model with conventional training (@fig-mnist-vanilla). In fact, this is not even using *ECCo+* and still showing better results than the best results we achieved in our AAAI paper for JEM ensembles.

:::

![Counterfactual images for *MLP* with counterfactual training. The underlying generator, *ECCo*, aims to generate counterfactuals that are faithful to the model [@altmeyer2024faithful].](/paper/figures/mnist_mlp.png){#fig-mnist}

![Counterfactual images for *MLP* with conventional training. The underlying generator, *ECCo*, aims to generate counterfactuals that are faithful to the model [@altmeyer2024faithful].](/paper/figures/mnist_mlp_vanilla.png){#fig-mnist-vanilla}

## Initial Grid Search {#sec-app-initial}


In [ ]:
results_dir = ENV["INITIAL_RUN_RESULTS"]
config_dir = ENV["INITIAL_RUN_CONFIG"]

For the initial round of experiments we 

### Generator Parameters

The hyperparameter grids for the first investigation of the effect of generator parameters are shown in @exr-gen-params-first-run-train and @exr-gen-params-first-run-eval.

::: {#exr-gen-params-first-run-train}

## Training Phase


In [ ]:
#| output: asis
dict = CTExperiments.from_toml(joinpath(config_dir,"gen_params.toml")) 
println(CTExperiments.dict_to_quarto_markdown(dict))

:::


::: {#exr-gen-params-first-run-eval}

## Evaluation Phase


In [ ]:
#| output: asis
dict = CTExperiments.from_toml(joinpath(results_dir, "gen_params/mlp/lin_sep/evaluation/evaluation_grid_config.toml"))
println(CTExperiments.dict_to_quarto_markdown(dict))

:::


In [ ]:
gen_params_dir = joinpath(results_dir, "gen_params/mlp")

#### Linearly Separable 

- **Energy Penalty** (@tbl-lin_sep-lambda_energy_exper): *ECCo* generally does yield better results than *Vanilla* for higher choices of the energy penalty (10,15) during training. *Generic* performs poorly accross the board. *Omni* seems to have an anchoring effect, in that it never performs terribly but also never as good as the best *ECCo* results. *REVISE* performs poorly across the board.
- **Cost** (@tbl-lin_sep-lambda_cost_exper): Results for all generators (except *Omni*) are quite bad, which can likely be attributed to extremely bad results for some choices of the **Energy Penalty** (results here are averaged). For *ECCo* and *Generic*, higher cost values generally lead to worse results.
- **Maximum Iterations**: No clear patterns recognizable, so it seems that smaller choices are ok. 
- **Validity**: *ECCo* almost always valid except for very low values during training and high values at evaluation time. *Generic* often has poor validity.
- **Accuracy**: Seems largely unaffected.


In [ ]:
df = CSV.read("$(gen_params_dir)/lin_sep/evaluation/results/ce/objective---lambda_energy_exper---lambda_energy_eval/plausibility_distance_from_target.csv", DataFrame)
df = groupby(df, Not(:run, :std, :mean, :lambda_energy_eval)) |> 
  gdf -> combine(gdf, :mean => (x -> [(mean(x),std(x))]) => [:value,:std]) |>
  df -> sort(df, [:lambda_energy_exper, :objective, :generator_type])

::: {#tbl-lin_sep-lambda_energy_exper}

::: {.content-hidden unless-format="pdf"}


In [ ]:
#| output: asis
get_table_inputs(df, "value"; alpha=0.9, byvars="lambda_energy_exper", backend=Val(:latex)) |>
    inputs -> tabulate_results(inputs; wrap_table=false)

:::

Results for Linearly Separable data by energy penalty.

:::

<!-- Cost -->


In [ ]:
df = CSV.read("$(gen_params_dir)/lin_sep/evaluation/results/ce/objective---lambda_cost_exper---lambda_energy_eval/plausibility_distance_from_target.csv", DataFrame)
df = groupby(df, Not(:run, :std, :mean, :lambda_energy_eval)) |> 
  gdf -> combine(gdf, :mean => (x -> [(mean(x),std(x))]) => [:value,:std]) |>
  df -> sort(df, [:lambda_cost_exper, :objective, :generator_type])

::: {#tbl-lin_sep-lambda_cost_exper}

::: {.content-hidden unless-format="pdf"}


In [ ]:
#| output: asis
get_table_inputs(df, "value"; alpha=0.9, byvars="lambda_cost_exper", backend=Val(:latex)) |>
    inputs -> tabulate_results(inputs; wrap_table=false)

:::

Results for Linearly Separable data by cost penalty.

:::


#### Moons

- **Energy Penalty** (@tbl-moons-lambda_energy_exper): *ECCo* consistently yields better results than *Vanilla*, except for very low choices of the energy penalty during training for which it performs abismal. *Generic* performs quite badly across the board for high enough choices of the energy penalty at evaluation time. *Omni* has small positive effect. *REVISE* performs poorly across the board.
- **Cost (distance penalty)**: *Generic* generally does better for higher values, while *ECCo* does better for lower values.
- **Maximum Iterations**: No clear patterns recognizable, so it seems that smaller choices are ok. 
- **Validity**: *ECCo* generally achieves full validity except for very low choices the energy penalty during training and high choices at evaluation time. *Generic* performs poorly for high choices of the energy penalty during evaluation.
- **Accuracy**: Largely unaffected although *ECCo* suffers a bit for very low choices the energy penalty during training. *REVISE* suffers a lot in general (around 10 percentage points).


In [ ]:
df = CSV.read("$(gen_params_dir)/moons/evaluation/results/ce/objective---lambda_energy_exper---lambda_energy_eval/plausibility_distance_from_target.csv", DataFrame)
df = groupby(df, Not(:run, :std, :mean, :lambda_energy_eval)) |> 
  gdf -> combine(gdf, :mean => (x -> [(mean(x),std(x))]) => [:value,:std]) |>
  df -> sort(df, [:lambda_energy_exper, :objective, :generator_type]) 

::: {#tbl-moons-lambda_energy_exper}

::: {.content-hidden unless-format="pdf"}


In [ ]:
#| output: asis
get_table_inputs(df, "value"; alpha=0.9, byvars="lambda_energy_exper", backend=Val(:latex)) |>
    inputs -> tabulate_results(inputs)

:::

Results for Moons data by energy penalty.

:::

#### Circles

- **Energy Penalty** (@tbl-circles-lambda_energy_exper): *ECCo* consistently yields better results than *Vanilla*, though primarily for low to medium choices of the energy penalty (<=5) during training. The same goes for *Generic*, which sometimes outperforms *ECCo* (for small energy penalty at evaluation time). *Omni* does alright for lower energy penalty at evaluation time, but loses out for higher choices. *REVISE* performs poorly across the board (except very low choices at evaluation time).
- **Cost (distance penalty)**: *ECCo* and *Generic* generally achieve the best results when no cost penalty is used during training. Both *Omni* and *REVISE* are largely unaffected.
- **Maximum Iterations**: *ECCo* consistently yields better results for higher numbers of iterations. *Generic* generally does best for a medium number (50). *Omni* is sometimes invalid (**???**).
- **Validity**: *ECCo* tends to outperform its *Vanilla* counterpart, though primarily for low to medium choices of the energy penalty (<=5) during training and evaluation. *Vanilla* typically worse across the board.
- **Accuracy**: Mostly unaffected, but *REVISE* again consistently some deterioration and *ECCo* deteriorates for high choices of energy penalty during training, reflecting other outcomes above.


In [ ]:
df = CSV.read("$(gen_params_dir)/circles/evaluation/results/ce/objective---lambda_energy_exper---lambda_energy_eval/plausibility_distance_from_target.csv", DataFrame)
df = groupby(df, Not(:run, :std, :mean, :lambda_energy_eval)) |> 
  gdf -> combine(gdf, :mean => (x -> [(mean(x),std(x))]) => [:value,:std]) |>
  df -> sort(df, [:lambda_energy_exper, :objective, :generator_type]) 

::: {#tbl-circles-lambda_energy_exper}

::: {.content-hidden unless-format="pdf"}


In [ ]:
#| output: asis
get_table_inputs(df, "value"; alpha=0.9, byvars="lambda_energy_exper", backend=Val(:latex)) |>
    inputs -> tabulate_results(inputs)

:::

Results for Circles data by energy penalty.

:::